<a href="https://colab.research.google.com/github/aaronbowman/Data_Tools/blob/nested_buttons/Consolidated_Data_Tools_(non_effect).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Top level entry point for uploading csvs
from google.colab import files
import io
import pandas as pd


#author_data = files.upload()
#outlet_data = files.upload

tags_set = files.upload()
#Read the tag data into a dataframe
tag_set = pd.read_csv(io.BytesIO(tags_set['BHE Tag Set.csv']), encoding ='latin1')

prior_article_data = files.upload()
#Read the previous time frames article data into a dataframe
prior_article_data = pd.read_csv(io.BytesIO(prior_article_data['BHE(Q3)New.csv']))
#Drop the last row of the dataframe as it has the 'Copyright PublicRelay 2022' string
prior_article_data.drop(prior_article_data.tail(1).index,inplace=True)

article_data = files.upload()
#Read the article data into a dataframe
article_data = pd.read_csv(io.BytesIO(article_data['BHE(Q4)New.csv']))
#Drop the last row of the dataframe as it has the 'Copyright PublicRelay 2022' string
article_data.drop(article_data.tail(1).index,inplace=True)




Saving BHE Tag Set.csv to BHE Tag Set.csv


Saving BHE(Q3)New.csv to BHE(Q3)New.csv


Saving BHE(Q4)New.csv to BHE(Q4)New.csv


In [17]:
# Create the needed 2D tags by merging our lists of data together there. Can swap these out for any other tagset from other accounts. Run this cell to make the fuction accessible 

# Create the 2D tags
parent_groups = tag_set['Parents'].dropna().tolist()
messages = tag_set['Messages'].dropna().values.tolist()



# Merge the 2D tags into a proper list and return that list. 
def topic_name():
    relational_tags = messages
    parent_tags = parent_groups
    tag_list = []
    
    for i in range(len(parent_tags)):
      for j in range(len(relational_tags)):
        tag_list.append(parent_tags[i] + ':' + relational_tags[j])
        
    return tag_list

# Use this to only generate the relevant 2D tags for a certain subsidary
# Use primarly when using nested drop-down menus to limit how much data needs to be crunched
def subsidary_tags_only(parent, message):
  
  output = parent + ':' + message

  return output

# Filter the dataframe so it only returns the 2D tags in the targeted dataframe
def relational_tags_only(df):

    # Import the topics and strip the dataframe in use of all unneeded columns
    topic = topic_name(effect, strategic_priorities, mission, mission_tags, strategic_tags)
    output_frame = df.filter(topic)

    return output_frame

# Pass a list as a an argument to amend the tag list and then filter the dataframe based on that
def relational_tags_plus(df, input_list):
    tag_list = topic_name()
    input_list = []
    complete_list = input_list + tag_list

    output_frame = df.filter(complete_list)

    return output_frame

def zero_safe_percent(prior, current):

  try:
    percent = (current/prior)*100
  except ZeroDivisionError:
    percent = 0
  return percent


In [4]:
# Group articles by tone and by day. Useful to generate graphs of specific tags/ tagsets and track them over time
from google.colab import files

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import io

# Correct the datetime of the loaded csv so that we can groupby datetime
def correct_datetime(df):
  df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
  return df

# Group the articles by article tone into rows by day
def article_tone_by_day(df):
  df = correct_datetime(df)
  # Filter the dataframe to make it a far more managebla size
  df.filter(items=['Date', 'Tone'])
  # Create the dateframe that we will export to
  output = pd.DataFrame(columns=['Date', 'Positive', 'Netural', 'Negative'])
  for j in range(len(df)):
    row = {}
    try:
      if pd.notna(df.at[j, 'Tone']):
        # We are adding a new row of data for each article with the correct values. Each row is a dict to be appended to the output dataframe
        if df.at[j, 'Tone'] == '+':
          row = {'Date': df.at[j, 'Date'], 'Positive' : 1, 'Netural' : 0, 'Negative' : 0}
        elif df.at[j, 'Tone'] == 'o':
          row = {'Date': df.at[j, 'Date'], 'Positive' : 0, 'Netural' : 1, 'Negative' : 0}
        elif df.at[j, 'Tone'] == '-':
          row = {'Date': df.at[j, 'Date'], 'Positive' : 0, 'Netural' : 0, 'Negative' : 1}
        
      output = output.append(other=row, ignore_index=True)
    except KeyError:
      print(f'No cases of Tone at: {j}')
      j += 1
  # Once all the rows are appended, we will groupby the datetime and sum the rows to give us a day by day count.
  output = output.groupby([output['Date'].dt.date]).sum()
  return output

# Group the articles by tag and the tone of that tag into rows by day
def tag_tone_by_day(df, target_tag):
  df = correct_datetime(df)

  # Filter the dataframe to make it a far more manageable size
  try:
    df.filter(items=['Date', target_tag], axis='columns')
  except KeyError:
      # If a tag doesn't exist zero it out at the first date in the main article dataframe
      print(f'No cases of tag {target_tag}')
      return {'Date': article_data.at[1, 'Date'], 'Positive' : 0, 'Netural' : 0, 'Negative' : 0} 

  # Create the dateframe that we will export to
  output = pd.DataFrame(columns=['Date', 'Positive', 'Netural', 'Negative'])
  output['Date'] = df['Date']

  try:
  
    output.loc[df[target_tag] == 1, 'Positive'] = 1
    output.loc[df[target_tag] == 0, 'Netural'] = 1
    output.loc[df[target_tag] < 0, 'Negative'] = 1

    # Once all the rows are appended, we will groupby the datetime and sum the rows to give us a day by day count.
    output = output.groupby([output['Date'].dt.date]).sum()
  except KeyError:
      # If a tag doesn't exist zero it out at the first date in the main article dataframe
      print(f'No cases of tag {target_tag}')
      return {'Date': article_data.at[1, 'Date'], 'Positive' : 0, 'Netural' : 0, 'Negative' : 0}
  return output

In [18]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets


def plot_tone_over_time(df):
  # Using matplotlib for consitancy, we plot the data from the article tone cells here. 
  fig = plt.figure(figsize=(10, 8), dpi=100)
  ax = fig.add_subplot(111)

  ax.plot(df.index, df['Positive'], c='g', label='Positive')
  ax.plot(df.index, df['Netural'], c='b', label='Netural')
  ax.plot(df.index, df['Negative'], c='r', label='Negative')
  plt.legend(loc='upper left');


# Upload the article data at the top and then it will be plottable here. This will call the groupby function and depdenacies
def tag_or_article_tone(parent_tag, message_tag):
  if parent_tag == 'All':
    plot_tone_over_time(article_tone_by_day(article_data))
  else:
    target_tag = subsidary_tags_only(parent_tag, message_tag)
    plot_tone_over_time(tag_tone_by_day(article_data, target_tag))

parent_drop_down = parent_groups
parent_drop_down.append('All')
parent_drop_down.sort()
message_drop_down = messages
message_drop_down.sort()

widgets.interact(tag_or_article_tone, parent_tag=parent_drop_down, message_tag = messages)

interactive(children=(Dropdown(description='parent_tag', options=('All', 'AltaLink', 'BHE', 'BHE GT&S', 'BHE R…

<function __main__.tag_or_article_tone>

In [19]:
# Prep and plot a linear regression model for each of the three articles tones over time
from scipy import stats
import matplotlib as mpl
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
import scipy.stats as sp

# Create a dummy varibale for time so that we can plot the linear regression models
def regression_dummy(df):
  # Assuming that the dataframe is grouped by a consistnacy time period and in order, this will provide a linear count as a dummy variable. Day 1 = 1, Day 2 = 2 etc..  
  for i in range(len(df)):
    df.at[df.index[i], 'Regression Dummy'] = i + 1
  return df

# This will return all the key datapoints of a linear regression model to report on it and plot
def linear_regression(data, target):

  # Target variable here should be a str telling us which of Positive/Neutral/Negative we are aiming for in this run
  y=np.array(data[target].dropna().values, dtype=float)
  x=np.array(data['Regression Dummy'].dropna().values, dtype=float)
  slope, intercept, r_value, p_value, std_err = stats.linregress(data['Regression Dummy'], data[target])

  # Calulate the y values for each point of x to facilate plotting
  xf = np.linspace(min(x),max(x),100)
  yf = (slope*xf)+intercept

  # Print the key values of our model so we can actually use it to assess if its valid and or useful
  print('Target value =', target, '\n', 'r = ', r_value, '\n', 'p = ', p_value, '\n', 's = ', std_err)
  return slope, intercept, r_value, p_value, std_err, xf, yf

def plot_over_time(df):
  fig = plt.figure(figsize=(10, 8), dpi=100)
  ax1 = fig.add_subplot(111)

  # Add the regression dummy variable to the df
  df = regression_dummy(df)

  # Time series is hard, so we need to plot with the regression dummy variable
  ax1.scatter(df['Regression Dummy'], df['Positive'], s=10, c='g', marker="s", label='Positive')
  ax1.scatter(df['Regression Dummy'], df['Netural'], s=10, c='b', marker="o", label='Netural')
  ax1.scatter(df['Regression Dummy'], df['Negative'], s=10, c='r', marker="x", label='Negative')
  # Plot the linear regression lines. Colours should be the same as the scatter plots for their tones
  slope, intercept, r_value, p_value, std_err, xf, yf = linear_regression(df, 'Positive')
  ax1.plot(xf, yf, label='Positive Linear fit', lw=3, c='g')
  slope, intercept, r_value, p_value, std_err, xf, yf = linear_regression(df, 'Netural')
  ax1.plot(xf, yf, label='Netural Linear fit', lw=3, c='b') 
  slope, intercept, r_value, p_value, std_err, xf, yf = linear_regression(df, 'Negative')
  ax1.plot(xf, yf, label='Negative Linear fit', lw=3, c='r')

  plt.legend(loc='upper left');


# Upload the article data at the top and then it will be plottable here. This will call the groupby function and depdenacies
def tag_or_article_linear_regression(parent_tag, message_tag):
  if parent_tag == 'All':
    plot_over_time(article_tone_by_day(article_data))
  else:
    target_tag = subsidary_tags_only(parent_tag, message_tag)
    plot_over_time(tag_tone_by_day(article_data, target_tag))

parent_drop_down = parent_groups
parent_drop_down.append('All')
parent_drop_down.sort()
message_drop_down = messages
message_drop_down.sort()

widgets.interact(tag_or_article_linear_regression, parent_tag=parent_drop_down, message_tag = messages)

interactive(children=(Dropdown(description='parent_tag', options=('All', 'All', 'AltaLink', 'BHE', 'BHE GT&S',…

<function __main__.tag_or_article_linear_regression>

In [ ]:
import pandas as pd
import io

def total_tag_tone(current_df, previous_df):

  
  tag_list = topic_name()
  output_frame = pd.DataFrame(columns=['Parent', 'Message', 'Tag', 'Current Total', 'Prior Total', 'Total Change', 'Current Positive', 'Prior Positive', 'Positive Change',
                               'Current Neutral', 'Prior Neutral', 'Netural Change', 'Current Negative', 'Prior Negative', 'Negative Change'])

  for i in range(len(tag_list)):
    try:
      current_working = tag_tone_by_day(current_df, tag_list[i])
      current_working = {'Tag' : tag_list[i], 'Current Positive' : current_working['Positive'].sum(axis = 0, skipna = True), 
                        'Current Neutral' : current_working['Netural'].sum(axis = 0, skipna = True),
                        'Current Negative' : current_working['Negative'].sum(axis = 0, skipna = True)}
      current_working['Current Total'] = current_working['Current Positive'] + current_working['Current Neutral'] + current_working['Current Negative']
      prior_working = tag_tone_by_day(previous_df, tag_list[i])


      prior_working = {'Tag' : tag_list[i], 'Prior Positive' : prior_working['Positive'].sum(axis = 0, skipna = True), 
                        'Prior Neutral' : prior_working['Netural'].sum(axis = 0, skipna = True),
                        'Prior Negative' : prior_working['Negative'].sum(axis = 0, skipna = True)}
      prior_working['Prior Total'] = prior_working['Prior Positive'] + prior_working['Prior Neutral'] + prior_working['Prior Negative']

      # Join the two dicts together
      output = {**current_working, **prior_working}
      
      # Check to see if total in both time preiods is 0. If so, there is no data there and we should just discard the row
      if output['Current Total'] == 0 and output['Prior Total'] == 0:
        print('No values in either item period for tag: {tag_list[i]}')
        i += 1
      # If there is data in at least one of the two rows then we should perform the calculations
      else:
        output['Total Change'] = zero_safe_percent(output['Prior Total'], output['Current Total'])
        output['Positive Change'] = zero_safe_percent(output['Prior Positive'], output['Current Positive'])
        output['Netural Change'] = zero_safe_percent(output['Prior Neutral'], output['Current Neutral'])
        output['Negative Change'] = zero_safe_percent(output['Prior Negative'], output['Current Negative'])
        
        # Determine which parent tag it is and add to the column. This is to make it easier to filter by
        for i in range(len(parent_groups)):
          if parent_groups[i] in output['Tag']:
            output['Parent'] = parent_groups[i]
        
        # Determine which message tag it is and add to the column. This is to make it easier to filter by
        for i in range(len(messages)):
          if messages[i] in output['Tag']:
            output['Message'] = messages[i]

        output_frame = output_frame.append(other=output, ignore_index=True)
    except AttributeError as e:
      print(e)
      i += 1

  return output_frame
def execute_tag_tone(current, prior):
  data = total_tag_tone(current, prior)
  data.to_csv(path_or_buf=r'tag_tone.csv')
  
execute_tag_tone(article_data, prior_article_data)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in long_scalars


No values in either item period for tag: {tag_list[i]}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: divide by zero encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: divide by zero encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: divide by zero encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: divide by zero encountered in long_scalars
/usr/local/l

No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in long_scalars


...

No values in either item period for tag: {tag_list[i]}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: divide by zero encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in long_scalars


No cases of tag BHE GT&S:5l. Security
'int' object has no attribute 'sum'
No cases of tag BHE GT&S:5m. Energy Efficienc
'int' object has no attribute 'sum'
No cases of tag BHE GT&S:5n. Other
'int' object has no attribute 'sum'


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in long_scalars


No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No values in either item period for tag: {tag_list[i]}
No cases of tag MidAmerican Services:5l. Security
'int' object has no attribute 'sum'
No cases of tag MidAmerican Services:5m. Energy Efficienc
'int' object has no attribute 'sum'
No cases of tag MidAmerican Services:5n. Other
'int' object has no attribute 'sum'
No values in either item period for tag: {tag_list[i]}
No cases of tag MidAme

In [ ]:
import pandas as pd
import io

def proactive_reactive_percent(total, proactive, reactive):

  try:
    proactive_percent = (proactive/total)*100
  except ZeroDivisionError:
    proactive_percent = 0
  
  try:
    reactive_percent = (reactive/total)*100
  except ZeroDivisionError:
    reactive_percent = 0
  
  return proactive_percent, reactive_percent



def proactive_reactive_by_tag(df):

    #Call the topic list that we will need to filter by
    topic = topic_name()
    df.filter(items=topic, axis='columns')

    #Create the output dataframe (not called output as that name is used in the main function loop)
    dataframe = pd.DataFrame(columns=['Tag', 'Total', 'Proactive', 'Proactive %', 'Reactive', 'Reactive %',
              'Proactive:Positive', 'Proactive:Neutral', 'Proactive:Negative',
              'Reactive:Positive', 'Reactive:Neutral', 'Reactive:Negative'])

    for i in range(len(topic)):
        count = 0
        proactive_positive_count = 0
        proactive_neutral_count = 0
        proactive_negative_count = 0
        reactive_positive_count = 0
        reactive_neutral_count = 0
        reactive_negative_count = 0
        for j in range(len(df)):
            try:
                if pd.notna(df.at[j, topic[i]]):
                  #Count refereces the complete total of articles, both primary and syndicates 
                    count += 1
                    if df.at[j, 'TVA:Proactive'] == 1:
                        proactive_positive_count += 1
                    elif df.at[j, 'TVA:Proactive'] < 0:
                        proactive_negative_count += 1
                    elif df.at[j, 'TVA:Proactive'] == 0:
                        proactive_neutral_count += 1

                    elif df.at[j, 'TVA:Reactive'] == 1:
                        reactive_positive_count += 1
                    elif df.at[j, 'TVA:Reactive'] < 0:
                        reactive_negative_count += 1
                    elif df.at[j, 'TVA:Reactive'] == 0:
                        reactive_neutral_count += 1

            except KeyError:
              #In the case where the tag is not in use, 0 out all the values and add to the dataframe to ensure that we have some value to call on.
              output = {'Tag' : topic[i], 'Total' : 0, 'Proactive' : 0, 'Proactive %' : 0, 'Reactive' : 0, 
                  'Reactive %' : 0, 'Proactive:Positive' : 0, 'Proactive:Neutral' : 0, 'Proactive:Negative' : 0,
                 'Reactive:Positive' : 0, 'Reactive:Neutral' : 0, 'Reactive:Negative' : 0}
              dataframe = dataframe.append(other=output, ignore_index=True)
              i += 1
            

        proactive_total = proactive_positive_count + proactive_neutral_count + proactive_negative_count
        reactive_total = reactive_positive_count + reactive_neutral_count + reactive_negative_count

        proactive_percent, reactive_percent = proactive_reactive_percent(count, proactive_total, reactive_total)

        # Dict to append to the dataframe
        output = {'Tag' : topic[i], 'Total' : count, 'Proactive' : proactive_total, 'Proactive %' : proactive_percent, 'Reactive' : reactive_total, 
                  'Reactive %' : reactive_percent, 'Proactive:Positive' : proactive_positive_count, 'Proactive:Neutral' : proactive_neutral_count, 'Proactive:Negative' : proactive_negative_count,
                 'Reactive:Positive' : reactive_positive_count, 'Reactive:Neutral' : reactive_neutral_count, 'Reactive:Negative' : reactive_negative_count}
        dataframe = dataframe.append(other=output, ignore_index=True)

    return dataframe
def execute_proactive_reactive_by_tag(df):
  data = proactive_reactive_by_tag(article_data)
  data.to_csv(path_or_buf=r'proactive_reactive_output.csv')